In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sea
%matplotlib inline

In [2]:
reek_columns =[
    'reek_total',
 'Attribute',
 'BooleanParameter',
 'ClassVariable',
 'ControlParameter',
 'DataClump',
 'DuplicateMethodCall',
 'FeatureEnvy',
 'InstanceVariableAssumption',
 'IrresponsibleModule',
 'LongParameterList',
 'LongYieldList',
 'ManualDispatch',
 'ModuleInitialize',
 'NestedIterators',
 'NilCheck',
 'PrimaDonnaMethod',
 'RepeatedConditional',
 'SubclassedFromCoreClass',
 'TooManyConstants',
 'TooManyInstanceVariables',
 'TooManyMethods',
 'TooManyStatements',
 'UncommunicativeMethodName',
 'UncommunicativeModuleName',
 'UncommunicativeParameterName',
 'UncommunicativeVariableName',
 'UnusedParameters',
 'UtilityFunction',
 'delta_Attribute',
 'delta_BooleanParameter',
 'delta_ClassVariable',
 'delta_ControlParameter',
 'delta_DataClump',
 'delta_DuplicateMethodCall',
 'delta_FeatureEnvy',
 'delta_InstanceVariableAssumption',
 'delta_IrresponsibleModule',
 'delta_LongParameterList',
 'delta_LongYieldList',
 'delta_ManualDispatch',
 'delta_ModuleInitialize',
 'delta_NestedIterators',
 'delta_NilCheck',
 'delta_PrimaDonnaMethod',
 'delta_RepeatedConditional',
 'delta_SubclassedFromCoreClass',
 'delta_TooManyConstants',
 'delta_TooManyInstanceVariables',
 'delta_TooManyMethods',
 'delta_TooManyStatements',
 'delta_UncommunicativeMethodName',
 'delta_UncommunicativeModuleName',
 'delta_UncommunicativeParameterName',
 'delta_UncommunicativeVariableName',
 'delta_UnusedParameters',
 'delta_UtilityFunction',
 'delta_reek_total']
lizard_columns = [
 'avg_ccn',
 'avg_nloc',
 'avg_token',
 'fun_count',
 'fun_rate',
 'nloc_rate',
 'warning_count',
 'delta_avg_ccn',
 'delta_avg_nloc',
 'delta_avg_token',
 'delta_fun_count',
 'delta_fun_rate',
 'delta_warning_count',

]

rubocop_columns = [
 'delta_lint_',
 'delta_lint_duplicate',
 'delta_metrics_abcsize',
 'delta_metrics_blocklength',
 'delta_metrics_blocknesting',
 'delta_metrics_classlength',
 'delta_metrics_cyclomaticcomplexity',
 'delta_metrics_linelength',
 'delta_metrics_methodlength',
 'delta_metrics_modulelength',
 'delta_metrics_parameterlists',
 'delta_metrics_perceivedcomplexity',
 'delta_nloc',
 'delta_nloc_rate',
 'delta_style_',
 'delta_total',
        'delta_performance_',
    'delta_security_',
    'delta_metrics_',
 'lint_',
 'lint_duplicate',
 'metrics_abcsize',
 'metrics_blocklength',
 'metrics_blocknesting',
 'metrics_classlength',
 'metrics_cyclomaticcomplexity',
 'metrics_linelength',
 'metrics_methodlength',
 'metrics_modulelength',
 'metrics_parameterlists',
 'metrics_perceivedcomplexity',
 'style_',
    'performance_',
    'security_',
    'metrics_',
 'total',
 ]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import binarize
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif, mutual_info_classif

/Users/terry/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
%%writefile learner.py
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import binarize
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif, mutual_info_classif, f_regression, SelectPercentile

class RubygemLearner(object):
    def __init__(self, fn, exclude_features=[]):
        with open(fn) as f:
            data = json.load(f)
        self.data = data["data"]
        self.spec = data["spec"]
        self.df = pd.io.json.json_normalize(self.data)
        self.features = self.exclude_unwanted_features(exclude_features)
        self._rename_columns()
        self.df = self.df[self.df["avg_nloc"]<20]
            
    def exclude_unwanted_features(self, exclude_features):
        return [k for k, v in self.data[0]['stat'].items() if k not in exclude_features]
        
    def remove_too_small_changes(self):
        try:
            df = df.groupby('name').filter(lambda x: x['delta_nloc'].sum() >2000)
        except:
            pass
    
    def summary(self):
        print(self.spec)
        print('nb_records:', len(self.df))
        print("abandoned:", (self.df['label'] != 'maintained').sum())
        
    def feature_evaluation(self):
        X = self.df[self.features]
        y = (self.df.label == 'maintained').values.astype(int)
        X_new = SelectKBest(f_classif, k=10).fit(X, y)
        feature_list = np.array([X_new.scores_.astype(float), self.features]).T
        feature_list=[[float(x[0]), x[1]] for x in feature_list]
        return sorted(feature_list, key=lambda a_entry: a_entry[0], reverse=True )
        
    def _rename_columns(self):
        old_new_cols = {}
        for name in self.df.columns:
            if name.startswith('stat'):
                old_new_cols[name] = name.split('.')[1]
        self.df.rename(columns=old_new_cols, inplace=True)
        
    def my_train_test_split(self, test_size=0.3):
        Xgems = self.df[["name"]].drop_duplicates()
        X_train_gems, X_test_gems = train_test_split(Xgems, test_size=test_size)
        X_train = self.df[self.df['name'].isin(X_train_gems['name'])]
        y_train = (X_train.label == 'maintained').values.astype(int)
        X_train = X_train[self.features]
        dftest = self.df[self.df['name'].isin(X_test_gems['name'])]

        return X_train, y_train, dftest

class Trainer(object):
    def __init__(self, learner):
        self.learner = learner
        
    def predict_sequence(self, estimator, df, name, label, threshold=0.5):
        X = df[df['name']==name][df['label']==label][self.learner.features]
        y_pred = estimator.predict(X)
        return 0 if np.mean(y_pred) < threshold else 1

    def test_model(self, model, dftest):
        estimator = model
        testers = dftest[["name", "label"]].drop_duplicates()
        y_test = (testers.label == 'maintained').values.astype(int)
        y_pred = [self.predict_sequence(estimator, dftest, t[0], t[1]) for t in testers.values]
        #print(y_pred)
        #print(y_test)
        #print(metrics.classification_report(y_test, y_pred))
        return metrics.accuracy_score(y_test, y_pred)

    def train_and_test(self, model):
        accuracies = []
        for i in range(5):
            X_train, y_train, test = self.learner.my_train_test_split(test_size=0.3)
            model.fit(X_train, y_train)
            accuracies.append(self.test_model(model, test))
        return np.mean(accuracies)
    


Overwriting learner.py


In [5]:
from learner import RubygemLearner, Trainer

In [6]:
import json

fn = 'RubygemDigger--Steps--GenerateJsonForLastVersions--2.data.json'
fn = 'RubygemDigger--Steps--GenerateJsonForAllVersions--1.data.json.all.json'

learner = RubygemLearner(fn, reek_columns + rubocop_columns)
learner.summary()
print(learner.feature_evaluation())

{'version': 1, 'description': 'most data', 'min_number_of_gems': 12, 'min_months': 10, 'min_nloc': 2000, 'min_months_good': 20, 'history_months': 10, 'stopped_time_point': '2015-01-01 00:00:00 UTC', 'ignored_months_for_good': 10}
nb_records: 9968
abandoned: 3819
[[53.426880023707774, 'warning_count'], [40.59749253101271, 'fun_rate'], [40.20437310685042, 'fun_count'], [22.793168028098954, 'avg_ccn'], [20.83904074719494, 'nloc'], [17.904624546236967, 'nloc_rate'], [7.652223163799252, 'avg_nloc'], [3.286718520939363, 'delta_warning_count'], [3.0333366370115975, 'avg_token'], [2.7765661849735404, 'delta_avg_ccn'], [0.37218105669617146, 'delta_fun_rate'], [0.21933111326728505, 'delta_fun_count'], [0.06924892050043378, 'delta_avg_nloc'], [0.0011524377996549662, 'age'], [0.0007150614375141778, 'delta_avg_token']]


# Basic building blocks

In [7]:
trainer = Trainer(learner)

In [8]:
def label_counts_summary(data, label='unknow'):
    N = len(data)
    N_ones = data.sum()
    print('''
label: {}
nb. of observations: {}
nb. of ones (good gems): {}
frequency: {:.3f}
    '''.format(label, N, N_ones, 1.0 * N_ones / N))

# Classify using LogisticRegression

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    #BaggingClassifier(grid.best_estimator_, max_samples=0.5, max_features=0.5),
    BaggingClassifier(DecisionTreeRegressor()),
    BaggingClassifier(LogisticRegression()),
    BaggingClassifier(KNeighborsClassifier(n_neighbors=15)),
    RandomForestClassifier(n_estimators=20),
    AdaBoostClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=4, random_state=0),
    LinearSVC(loss='l2', penalty="l2",dual=False, tol=1e-4),
    MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1)
]


In [10]:
def evaluate_models(trainer, models):
    results = []
    for model in models:
        print("*" * 50)
        print("Class:", type(model).__name__)
        precision = trainer.train_and_test(model)
        print("Parameters:", model.get_params())
        print("Precision", precision)
        results.append(precision)
    return results


In [ ]:
import json

def evaluate_specs(trainer, spec_type_name):
    results = []
    for spec_id in [1,3,4,5,7,8]:    
        fn = '%s--%d.data.json' % (spec_type_name, spec_id)
        results.append(evaluate_models(trainer, models))

    for row in results:
        for n in row:
            print("|", "%4.3f" % n, end='')
        print("|")
    return results



In [ ]:
learner = RubygemLearner(fn)
trainer = Trainer(learner)
result_last = evaluate_specs(trainer, 'RubygemDigger--Steps--GenerateJsonForLastVersions')

**************************************************
Class: LogisticRegression


/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]


Parameters: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 100, 'multi_class': 'ovr', 'n_jobs': 1, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Precision 0.623015770379
**************************************************
Class: DecisionTreeClassifier


/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]


Parameters: {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_split': 1e-07, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}
Precision 0.550826857234
**************************************************
Class: BaggingClassifier


...

/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.

Parameters: {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_split': 1e-07, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 20, 'n_jobs': 1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}
Precision 0.61426070776
**************************************************
Class: AdaBoostClassifier


/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]
/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.

Parameters: {'algorithm': 'SAMME.R', 'base_estimator': None, 'learning_rate': 1.0, 'n_estimators': 100, 'random_state': None}
Precision 0.604407508755
**************************************************
Class: GradientBoostingClassifier


/Users/terry/git/rubygem_digger/notebook/learner.py:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X = df[df['name']==name][df['label']==label][self.learner.features]


In [ ]:
result_all = evaluate_specs(trainer, 'RubygemDigger--Steps--GenerateJsonForAllVersions')

In [ ]:
from itertools import chain
def print_results(results):
    for row in results:
        for n in row:
            print("|", "%4.3f" % n, end='')
        print("|", "%4.3f" % (sum(row)/len(row)), '|')
    for c in range(len(results[0])):
        col = [x[c] for x in results]
        print("|", "%4.3f" % (sum(col)/len(col)), end='')
    a = list(chain(*results))
    print("|", "%4.3f" % (sum(a)/len(a)), '|')



In [ ]:
print_results(result_last)

In [ ]:
print_results(result_all)

# Classify using kNN

In [ ]:
nb_range = np.arange(1, 31)
a_scores = []

for nb in nb_range:
    model = KNeighborsClassifier(n_neighbors=nb)
    a_scores.append(trainer.train_and_test(model))
    
plt.plot(nb_range, a_scores, 'o-')
plt.ylabel('Testing accuracy')
plt.xlabel('k neighbours')

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']
param_grid = dict(n_neighbors=k_range, weights=weight_options)

knn = KNeighborsClassifier()

grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid.fit(X, y)


# examine the best model
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

# Ensemble

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
from sklearn.ensemble import BaggingClassifier
print(train_and_test(BaggingClassifier(grid.best_estimator_, max_samples=0.5, max_features=0.5), df))

In [ ]:
print(train_and_test(BaggingClassifier(DecisionTreeRegressor()), df))

In [ ]:
print(train_and_test(BaggingClassifier(LogisticRegression()), df))

In [ ]:
print(train_and_test(BaggingClassifier(KNeighborsClassifier(n_neighbors=15)), df))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
print(train_and_test(RandomForestClassifier(n_estimators=20), df))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
print(train_and_test(AdaBoostClassifier(n_estimators=100), df))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
print(train_and_test( GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=4, random_state=0), df))

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC(loss='l2', penalty="l2",dual=False, tol=1e-4)
print(train_and_test(model, df))

In [ ]:
model.decision_function

In [ ]:
from sklearn.neural_network import MLPClassifier
print(train_and_test(MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1), df))